In [23]:
#import av
import numpy as np
import math
from skimage.morphology import *
from skimage.feature import *
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from operator import attrgetter
import seaborn as sns
from operator import attrgetter
from itertools import chain
from skimage import measure
from skimage import morphology
from skimage.util import invert
from skimage.filters import *
from scipy import ndimage
from glob import glob
from skvideo.io import vread
"""Currently works on a single frame. Later, will add a loop that can work on all frames."""

'Currently works on a single frame. Later, will add a loop that can work on all frames.'

In [17]:
# take largest isodata image region, and return aclassification
def sideify(cons_of_max):    
    if cons_of_max == 3:
        return "c"
    elif cons_of_max == 2:
        return "t"
    else:
        return "ucf"
    
def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    connections = 0
    for blob2 in bloblist:
        cv2 = (blob2[0], blob2[1]) # center of blob2
        if vector_dist(cv, cv2) <= 2*radius: # blob centers closer than diameter of one blob
            connections += 1
        else:
            pass
    return connections

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
        return l1[2]        

#from pair of labelled, thresholded images, return an image category (of three)

#from pair of labelled,thresholded images, return the x/y sign or x/y magnitude for the transparent dot


In [18]:
connect_test = connect((0, 0),2, [(0, 3, 1), (3, 3, 1.1), (0, 0, 5), (9, 9, 100)]) #2
vector_dist_test = [vector_dist((0, 0), (3, 4)), vector_dist((1,1,0), (1,7,8))]
third_item_test = third_item([0, "99", "hello", [55, 67]])
print(connect_test, vector_dist_test , third_item_test )

2 [5.0, 10.0] hello


In [19]:
"Filtering broken ad oddframes"
"USING LI THRESHOLD"
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(framelist, framesetname): 
    # returns a new list, different from the orginal
    fll = len(framelist)
    m_lab = measure.label
    m_rop = measure.regionprops
    expected_blobs = 4
    expected_connections = 5
    
    filtrate_prop =[]
    passes = []
    all_props = []
    blobset = [] #labelled imgs
    for i in range(fll):
       # ---setup image and detect blobs ----------------
        if framesetname == 'broken' or framesetname == 'odd_s':
            framesetname = 'ucf'
        frame = framelist[i]
        thresh_img = frame > threshold_isodata(frame)# binary image
        highlight = morphology.erosion(morphology.opening(thresh_img, square(7)),
                                        square(2))
        img = np.copy(frame)
        img[highlight==0] = 0
        blobs =skimage.feature.blob_doh(img, min_sigma = 6, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.75)
        blobset.append(blobs)
        
        num_connections = 0
        num_blobs = len(blobs)
        
        for i in range(num_blobs-1):
            blob = blobs[i]
            radius = blob[2]
            center_vec = (blob[0], blob[1])
            num_connections += connect(center_vec, radius, blobs[i+1::])
        #----------------------filter------------------------- can use actual filter
        
        properties = []
        max_blob = max(blobs, key = third_item)
        cons_of_max = connect((max_blob[0],max_blob[1]),max_blob[2], blobs)
        properties = [num_blobs, num_connections, cons_of_max]
        all_props.append(properties)
            
        test = (num_blobs == expected_blobs
                and num_connections == expected_connections 
               )
        if test:  
            if (cons_of_max == 3 or cons_of_max == 2):
                #-----------------------------classify pt 1---------------------------------------
                clas = sideify(cons_of_max)
                filtrate_prop.append(properties)
            else:
                clas = 'ucf'
        else:
            clas = 'ucf'
        if clas == framesetname:
            passes.append(properties)
            
    return [filtrate_prop, passes, all_props, blobset]






In [20]:
filenames = filename = glob("./tchakamau/trn*/*[1][0]v*[0,6,2].avi")#glob("./tchakamau/New_setup/*.avi")# glob("./tchakamau/New_setup/45681_10Vpp_25.avi") #filenames = 
print(filenames)
num_shape_cats = 4
Pass_ratios = np.zeros((len(filenames), num_shape_cats))



"""TEST FRAME RANGES FROM VIDEOs"""
phantom_frames = {"t" : list(range(37949, 30697, -25)),
                  "c" :list(range(20951, 20172, -3)), 
                 "broken" :list(range(21927, 21593, -1)) ,
                 "odd_s" :list(range(21593,21577, -1)) }
phantom_frames_2 = {"t" : list(range(26391, 22129, -25)),
                  "c" :list(range(21152, 20607, -1 )), 
                 "broken" :list(range(21979, 21309, -2)) ,
                 "odd_s" :list(chain(range(22115, 22080, -1), range(22017,21980, -1), range(21302, 21272, -1))) }
phantom_frames_3 = {"t" :list(range(30104, 29591, -1)),
                  "c" :list(range(30816, 30375, -2)), 
                 "broken" :list(range(30340, 30288, -1)) ,
                 "odd_s" :[]}

"""phantom_frames = {"t" : list(range(13268, 13529)),
                  "c" :list(range(16740, 16951)), 
                 "broken" :list(range(14224, 14623, 2)) ,
                 "odd_s" :list(range(14001,14044)) }
phantom_frames_2 = {"t" : list(range(21985, 22175)),
                  "c" :list(range(18192, 18400)), 
                 "broken" :list(range(20904, 21625, 4)) ,
                 "odd_s" :list(chain(range(20360, 20381), range(20386, 20420))) }
phantom_frames_3 = {"t" :list(range(3266, 3491)),
                  "c" :list(range(3574, 3597)), 
                 "broken" :list(range(3686, 4209, 3)) ,
                 "odd_s" :list(chain(range(3521, 3538), range(3668, 3680))) }
phantom_frames_4 = {"t" :list(range(9361, 9709, 2)),
                  "c" :list(range(12800,13578, 3)), 
                 "broken" :list(range(11057, 11482, 2)) ,
                 "odd_s" :[] }"""
phantom_frame_sets = [phantom_frames,phantom_frames_2,phantom_frames_3]
trigger_frames = [37949, 26391,30816]#[13268, 18192, 2964, 8535]


['./tchakamau\\Trnansparentd\\45400_10Vpp_0.avi', './tchakamau\\Trnansparentd\\45403_10Vpp_6.avi', './tchakamau\\Trnansparentd\\45405_10Vpp_2.avi']


In [24]:
t_frames = []
tvals  = []
c_frames = []
cvals = []

odd_s_frames = []
oddsvals = []
broken_frames = []
brokenvals = []


framesets = [t_frames, c_frames,  odd_s_frames, 
             broken_frames]
frametypevals = [tvals, cvals,  oddsvals, brokenvals]
framesetnames = ["t", "c", "odd_s", "broken"]
filtrates_props = []
passes = []
i_props = []
t_props = []
    
for vidnum in range(len(filenames)):
    frames = vread(filenames[vidnum])
    frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    print(len(frame_vid))
    
    
    for i in range(len(framesetnames)):
        typevals = phantom_frame_sets[vidnum].get(framesetnames[i])
        frametypevals[i].extend(typevals)
        #print(len(typevals), trigger_frames[vidnum], typevals[-1], typevals[-1]-trigger_frames[vidnum])
        framesets[i].extend([(frame_vid[trigger_frames[vidnum]-x], x) for x in typevals])


37950
22303
26935


In [26]:
filtrates_props = []
passes = []
a_props = []
p_rats = []
limgs = []
for i in range(len(framesetnames)):
    x = total_threshold_filter([y[0] for y in framesets[i]], framesetnames[i])
    filtrates_props.append(x[0])
    passes.append(x[2])
    #all_props.append(x[2])
    #print(framesets[i])

    orig = len(frametypevals[i])
    fil = len(x[1])
    print(orig, len(filtrates_props[i]), len(x[1]))
    print(framesetnames[i], "_vid")
    print("original_vid: ", orig)
    print("filtrate: ", fil)
    if orig != 0:
        print("pass ratio: ", fil*100/orig)
        p_rats.append(fil*100/orig)
    print("\n")
        

    a_props.append(x[2])
    limgs.append(x[3])
Pass_ratios = p_rats
Filtrates_props = filtrates_props
st = framesets
np.save('shape_testing.npy',st)
print(len(a_props), len(filtrates_props), len(passes))

975 0 0
t _vid
original_vid:  975
filtrate:  0
pass ratio:  0.0


1026 0 0
c _vid
original_vid:  1026
filtrate:  0
pass ratio:  0.0


118 0 118
odd_s _vid
original_vid:  118
filtrate:  118
pass ratio:  100.0


721 0 721
broken _vid
original_vid:  721
filtrate:  721
pass ratio:  100.0


4 4 4


"""FINDING CORRECT THRESHOLDS FOR SEPARATING BROKEN AND ODD SHAPED FRAMES"""

# use area information from properties
Inertias_list = []
for i in range(num_shape_cats):
    x = [h.inertia_tensor[0, 0] + h.inertia_tensor[1,1] for h in I_thresh_props[i]]
    Inertias_list.append(x) #list of areas for every category
print("Inertia averages: ")
[print("%s : %.f " %(framesetnames[i], np.average(Inertias_list[i]))) for i in range(num_shape_cats)]
print("Inertia variances: ")
[print("%s : %.f " %(framesetnames[i], np.var(Inertias_list[i]))) for i in range(num_shape_cats)]

fig, ax = plt.subplots(figsize=(18,8))
plt.title("Frequency distribution: inertia tensor_vid1")
plt.xlabel("inertia tensor")
plt.ylabel("Frequency")
for j in range(0, len(framesets)):
    x = Inertias_list[j]
    sns.distplot(x, hist = False, rug=True, label = framesetnames[j])
#--------

#------------------------

fig, ax = plt.subplots(figsize=(18,8))
plt.title("Frequency distribution: inertia tensor_vid1")
plt.xlabel("inertia tensor")
plt.ylabel("Frequency")
for j in range(0, len(framesets)):
    x = Inertias_list[j]
    sns.distplot(x, hist = False, rug=True, label = framesetnames[j])

propnames = ["convex_area", "minor_axis_length", "major_axis_length", "solidity", "eccentricity"]
for k in range(0, len(propnames)):
    klist = []
    for i in range(num_shape_cats):
        x = [getattr(h, propnames[k]) for h in I_thresh_props[i]]
        klist.append(x) #list of areas for every category
    print("%s averages: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.average(klist[i]))) for i in range(num_shape_cats)]
    print("%s variances: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.var(klist[i]))) for i in range(num_shape_cats)]    
    fig, ax = plt.subplots(figsize=(18,8))
    #plt.xticks(np.arange(0, 2501, 75))
    plt.title("Frequency distribution: " + propnames[k])
    plt.xlabel(propnames[k])
    plt.ylabel("Frequency")
    for j in range(0, len(framesets)):
        x = klist[j]
        sns.distplot(x, hist = False, rug=True, label = framesetnames[j])



"""Broken objects are filtered out by their area and we then move on to identifying odd shaped objects 
    from the remaining frames"""
framesets.remove(broken_frames)
framesetnames.remove("broken")

In [ ]:
"""STILL RESULTS OF FILTERING"""
Inertias_list = []
for i in range(num_shape_cats):
    x = [h[0].inertia_tensor[0, 0] + h[0].inertia_tensor[1,1] for h in Filtrates_props[i]]
    Inertias_list.append(x) #list of areas for every category
print("Inertia averages: ")
[print("%s : %.f " %(framesetnames[i], np.average(Inertias_list[i]))) for i in range(num_shape_cats)]
print("Inertia variances: ")
[print("%s : %.f " %(framesetnames[i], np.var(Inertias_list[i]))) for i in range(num_shape_cats)]

fig, ax = plt.subplots(figsize=(18,8))
plt.title("Frequency distribution: inertia tensor_vid")
plt.xlabel("inertia tensor")
plt.ylabel("Frequency")
for j in range(0, len(framesets)):
    x = Inertias_list[j]
    sns.distplot(x, hist = False, rug=True, label = framesetnames[j])

propnames = ["convex_area", "minor_axis_length", "major_axis_length", "solidity"]
for k in range(0, len(propnames)):
    klist = []
    for i in range(num_shape_cats):
        x = [getattr(h[0], propnames[k]) for h in Filtrates_props[i]]
        klist.append(x) #list of areas for every category
    print("%s averages: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.average(klist[i]))) for i in range(num_shape_cats)]
    print("%s variances: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.var(klist[i]))) for i in range(num_shape_cats)]    
    fig, ax = plt.subplots(figsize=(18,8))
    #plt.xticks(np.arange(0, 2501, 75))
    plt.title("Frequency distribution: " + propnames[k])
    plt.xlabel(propnames[k])
    plt.ylabel("Frequency")
    for j in range(0, len(framesets)):
        x = klist[j]
        sns.distplot(x, hist = False, rug=True, label = framesetnames[j])


In [ ]:
"""Pt3. Separating variosu shapes
"""

In [ ]:
"SEE HOW THE ISODATA FILTER TREATS FRAMES - UNFILTERED"                               

In [ ]:
"""PTINTING ISODATA THRESHOLDS"""
propnames = ["area", "extent"]
for k in range(0, len(propnames)):
    klist = []
    for i in range(num_shape_cats):
        x = [getattr(h[1], propnames[k]) for h in Filtrates_props[i]]
        klist.append(x) #list of areas for every category
    print("%s averages: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.average(klist[i]))) for i in range(num_shape_cats)]
    print("%s variances: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.var(klist[i]))) for i in range(num_shape_cats)]    
    fig, ax = plt.subplots(figsize=(18,8))
    #plt.xticks(np.arange(0, 2501, 75))
    plt.title("Frequency distribution: " + propnames[k])
    plt.xlabel(propnames[k])
    plt.ylabel("Frequency")
    for j in range(0, len(framesets)):
        x = klist[j]
        sns.distplot(x, hist = False, rug=True, label = framesetnames[j])
        

# most oddly shaped frames have elongated shapes, and the mnor axis length is below 40, except for one (of one partcular type)
# where the pixels for a 'T' shape; this, while more rounded, has a huge convex hull. All frames wth convex hull area 
# over 2500 (very conservatve estimate) areoddly shaped (transition frames). 
# Together the criteria select for frames not in transition.
#16766 is misclassified, but it isvery like a normally shaped t_l on most measures, so maybe it should be counted as one for this program

In [ ]:
"""FINDING CORRECT THRESHOLDS FOR SHAPE DISCRIMINATION"""

"""STILL RESULTS OF FILTERING"""
Inertias_list = []
for i in range(num_shape_cats):
    x = [h.inertia_tensor[0, 0] + h.inertia_tensor[1,1] for h in T_thresh_props[i]]
    Inertias_list.append(x) #list of areas for every category
print("Inertia averages: ")
[print("%s : %.f " %(framesetnames[i], np.average(Inertias_list[i]))) for i in range(num_shape_cats)]
print("Inertia variances: ")
[print("%s : %.f " %(framesetnames[i], np.var(Inertias_list[i]))) for i in range(num_shape_cats)]

fig, ax = plt.subplots(figsize=(18,8))
plt.title("Frequency distribution: inertia tensor_vid")
plt.xlabel("inertia tensor")
plt.ylabel("Frequency")
for j in range(0, len(framesets)):
    x = Inertias_list[j]
    sns.distplot(x, hist = False, rug=True, label = framesetnames[j])

    
Inertias_list = []
for i in range(num_shape_cats):
    x = [h[1].inertia_tensor[0, 0] + h[1].inertia_tensor[1,1] for h in Filtrates_props[i]]
    Inertias_list.append(x) #list of areas for every category
print("Inertia averages: ")
[print("%s : %.f " %(framesetnames[i], np.average(Inertias_list[i]))) for i in range(num_shape_cats)]
print("Inertia variances: ")
[print("%s : %.f " %(framesetnames[i], np.var(Inertias_list[i]))) for i in range(num_shape_cats)]

fig, ax = plt.subplots(figsize=(18,8))
plt.title("Frequency distribution: inertia tensor_vid")
plt.xlabel("inertia tensor")
plt.ylabel("Frequency")
for j in range(0, len(framesets)):
    x = Inertias_list[j]
    sns.distplot(x, hist = False, rug=True, label = framesetnames[j])
    
    
    
    
propnames = ["convex_area", "minor_axis_length", "major_axis_length", "solidity", "eccentricity"]
for k in range(0, len(propnames)):
    klist = []
    for i in range(num_shape_cats):
        x = [getattr(h, propnames[k]) for h in T_thresh_props[i]]
        klist.append(x) #list of areas for every category
    print("%s averages: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.average(klist[i]))) for i in range(num_shape_cats)]
    print("%s variances: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.var(klist[i]))) for i in range(num_shape_cats)]    
    fig, ax = plt.subplots(figsize=(18,8))
    #plt.xticks(np.arange(0, 2501, 75))
    plt.title("Frequency distribution: " + propnames[k])
    plt.xlabel(propnames[k])
    plt.ylabel("Frequency")
    for j in range(0, len(framesets)):
        x = klist[j]
        sns.distplot(x, hist = False, rug=True, label = framesetnames[j])
        
    klist = []
    for i in range(num_shape_cats):
        x = [getattr(h[1], propnames[k]) for h in Filtrates_props[i]]
        klist.append(x) #list of areas for every category
    print("%s averages: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.average(klist[i]))) for i in range(num_shape_cats)]
    print("%s variances: " %propnames[k])
    [print("%s : %.f " %(framesetnames[i], np.var(klist[i]))) for i in range(num_shape_cats)]    
    fig, ax = plt.subplots(figsize=(18,8))
    #plt.xticks(np.arange(0, 2501, 75))
    plt.title("Frequency distribution: " + propnames[k])
    plt.xlabel(propnames[k])
    plt.ylabel("Frequency")
    for j in range(0, len(framesets)):
        x = klist[j]
        sns.distplot(x, hist = False, rug=True, label = framesetnames[j])
        
        


# don't use: extent, perimeter, minor axis lengths, solidity, eccetricity
# inertia: 300 - 360 br/bl, 350 - 415 tr/tl, 417/412 - bm
# mal: 56 - 63 br/bl, 63 - 70 tr/tl, 68 bm
# ecc: .63 - .78 br/bl, .76 - .83 tr/tl, .76 bm

# if solidity <= .76 and inertia > 400 => bm
# elif inertia >=355 and mal >= 63 => tl/tr
# elif inertia <= 355 and mal <= 63 => bl/br
# else unclassifiable

In [ ]:
"""DO A PCA ON EACH OF THE SETS AND PLOT SEPARATION. DISPLAY PRINCIPAL AXIS LOADINGS"""